# Load the Pretrained Model and the Dataset

In [1]:
import sys

sys.path.insert(0, "../..")
sys.path.insert(0, "../../../")

In [2]:
import paddle
import paddlenlp
from paddlenlp.transformers import ErnieForSequenceClassification, ErnieTokenizer

MODEL_NAME = 'ernie-1.0'

model = ErnieForSequenceClassification.from_pretrained(MODEL_NAME, num_classes=2)
tokenizer = ErnieTokenizer.from_pretrained(MODEL_NAME)

[2022-06-07 11:42:22,184] [    INFO] - Already cached /home/zhangshuai/.paddlenlp/models/ernie-1.0/ernie_v1_chn_base.pdparams
W0607 11:42:22.187671 22920 device_context.cc:447] Please NOTE: device: 0, GPU Compute Capability: 7.0, Driver API Version: 11.4, Runtime API Version: 10.2
W0607 11:42:22.193305 22920 device_context.cc:465] device: 0, cuDNN Version: 8.2.
[2022-06-07 11:42:29,432] [    INFO] - Already cached /home/zhangshuai/.paddlenlp/models/ernie-1.0/vocab.txt


In [3]:
from paddlenlp.datasets import load_dataset

DATASET_NAME = 'chnsenticorp'
train_ds, dev_ds, test_ds = load_dataset(DATASET_NAME, splits=["train", "dev", "test"])

# Prepare the Model
## Train the model

In [ ]:
from assets.utils import training_model

training_model(model, tokenizer, train_ds, dev_ds, save_dir=f'../../assets/{DATASET_NAME}-{MODEL_NAME}')


## Or Load the trained model

In [4]:
# Load the trained model.
!wget --no-check-certificate -c https://trustai.bj.bcebos.com/chnsenticorp-ernie-1.0.tar
!tar -xvf ./chnsenticorp-ernie-1.0.tar -C ../../assets/
!rm ./chnsenticorp-ernie-1.0.tar

state_dict = paddle.load(f'../../assets/{DATASET_NAME}-{MODEL_NAME}/model_state.pdparams')
model.set_dict(state_dict)

--2022-06-07 11:42:29--  https://trustai.bj.bcebos.com/chnsenticorp-ernie-1.0.tar
Resolving trustai.bj.bcebos.com (trustai.bj.bcebos.com)... 10.70.0.165
Connecting to trustai.bj.bcebos.com (trustai.bj.bcebos.com)|10.70.0.165|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 399595520 (381M) [application/x-tar]
Saving to: ‘chnsenticorp-ernie-1.0.tar’

100%[======================================>] 399,595,520 91.1MB/s   in 4.5s   

2022-06-07 11:42:34 (84.1 MB/s) - ‘chnsenticorp-ernie-1.0.tar’ saved [399595520/399595520]

chnsenticorp-ernie-1.0/
chnsenticorp-ernie-1.0/tokenizer_config.json
chnsenticorp-ernie-1.0/vocab.txt
chnsenticorp-ernie-1.0/model_state.pdparams
chnsenticorp-ernie-1.0/model_config.json


# See the prediciton results

In [5]:
from assets.utils import predict

test_data = [{'text': '没有光驱,重装Windows需要外接光驱,对于电脑新手会很麻烦(没什么人会用Linux吧)'}]

label_map = {0: 'negative', 1: 'positive'}

batch_size = 32

results = predict(
    model, test_data, tokenizer, label_map, batch_size=batch_size)

for idx, text in enumerate(test_data):
    print('Data: {} \t Lable: {}'.format(text, results[idx]))

Data: {'text': '没有光驱,重装Windows需要外接光驱,对于电脑新手会很麻烦(没什么人会用Linux吧)'} 	 Lable: negative


# Prepare for Interpretations

In [6]:
from functools import partial

from paddlenlp.data import Stack, Tuple, Pad

from assets.utils import create_dataloader, convert_example


batch_size = 32
max_seq_length = 128

trans_func = partial(
        convert_example,
        tokenizer=tokenizer,
        max_seq_length=max_seq_length,
        is_test=True,
    )
batchify_fn = lambda samples, fn=Tuple(
    Pad(axis=0, pad_val=tokenizer.pad_token_id),  # input
    Pad(axis=0, pad_val=tokenizer.pad_token_type_id),  # segment
): [data for data in fn(samples)]
train_data_loader = create_dataloader(
    train_ds,
    mode='train',
    batch_size=batch_size,
    batchify_fn=batchify_fn,
    trans_fn=trans_func,
    shuffle=False)

# Feature similarity Interpreter

In [7]:
from trustai.interpretation import FeatureSimilarityModel

# classifier_layer_name is the layer name of the last output layer
feature_sim = FeatureSimilarityModel(model, train_data_loader, classifier_layer_name='classifier')

Extracting feature for given dataloader, it will take some time...


In feature similarity, Similarity function supports `cos` , `dot` and `euc`, namely `Cosine simmilarity`, `Dot product` and `Euclidean distance`.

Using cosine simmilarity:

In [9]:
from assets.utils import create_dataloader_from_scratch, print_result

test_data = [{'text': '没有光驱,重装Windows需要外接光驱,对于电脑新手会很麻烦(没什么人会用Linux吧)'}]

# process text to model input
test_dataloader = create_dataloader_from_scratch(test_data, tokenizer)

sim_fn = "cos"
sample_num = 3
res = feature_sim.interpret(test_dataloader, sample_num=sample_num, sim_fn=sim_fn)

print_result(test_data, train_ds, res, data_name='chnsenticorp')


Extracting feature for given dataloader, it will take some time...
test data
text: 没有光驱,重装Windows需要外接光驱,对于电脑新手会很麻烦(没什么人会用Linux吧)	predict label: 0
examples with positive influence
text: Linux系统不太好用,平时习惯用Windows xp 系统,一下子用这个系统感觉很不习惯,建议开发或预装Windows xp系统.	gold label: 0	score: 0.9393996000289917
text: 1、机器较沉 2、VISTA用起来不习惯，且占系统较多 3、音频插口、右前侧的二个USB口在用鼠标时感觉手靠得太近了	gold label: 0	score: 0.9354583621025085
text: vista系统下也没有无线网卡驱动，用驱动精灵可解决。 机器稍有点重。 散热有待改进。	gold label: 0	score: 0.9348428249359131
examples with negative influence
text: “任务型教学”在我国外语教学界备受关注。国家教育部新《英语课程标准》将“倡导‘任务型’的教学途径，培养学生综合语言运用能力”写入教学建议。任务型教学被视为改革我国传统外语教学的良方。本书立足我国外语教学现状，全面分析了“任务型教学”的理论和实践基础、以实例说明“任务型教学”的具体操作步骤。为广大一线英语教师提供了教学和研究参考。	gold label: 1	score: -0.12422356754541397
text: 当美国发生次贷危机乃至影响全世界以后，对于应对危机，我们国内的绝大多数专家对此都异口同声，观点基本雷同，而且又莫衷一是，人云亦云，本书的作者以其独特的视觉和丰富的知识，在书中告诉我们这次危机的来龙去脉，我们国家应该以怎样的方式去直面这次危机，如何转危为安，化危为机；作为普通读者也能从书中领会到一些对自己有益的知识。读完这本书以后，你更能体会到一种不一样的思维，非常值得一读。	gold label: 1	score: -0.128561332821846
text: 我从06年开始几乎每月都有一次出差，山西很多酒店都

Using Dot product:

In [10]:
sim_fn = "dot"

res = feature_sim.interpret(test_dataloader, sample_num=sample_num, sim_fn=sim_fn )

print_result(test_data, train_ds, res, data_name='chnsenticorp')

Extracting feature for given dataloader, it will take some time...
test data
text: 没有光驱,重装Windows需要外接光驱,对于电脑新手会很麻烦(没什么人会用Linux吧)	predict label: 0
examples with positive influence
text: Linux系统不太好用,平时习惯用Windows xp 系统,一下子用这个系统感觉很不习惯,建议开发或预装Windows xp系统.	gold label: 0	score: 510.1300354003906
text: 装系统比较麻烦，建议找专业人员安装。 桌面图标太大，不太美观。	gold label: 0	score: 507.4688415527344
text: 摄像头似乎有点问题,音孔太紧,开机太慢,刚买蓝屏一次,太巨型,光驱太劣质,接口没全开	gold label: 0	score: 507.4252014160156
examples with negative influence
text: “任务型教学”在我国外语教学界备受关注。国家教育部新《英语课程标准》将“倡导‘任务型’的教学途径，培养学生综合语言运用能力”写入教学建议。任务型教学被视为改革我国传统外语教学的良方。本书立足我国外语教学现状，全面分析了“任务型教学”的理论和实践基础、以实例说明“任务型教学”的具体操作步骤。为广大一线英语教师提供了教学和研究参考。	gold label: 1	score: -65.31314086914062
text: 当美国发生次贷危机乃至影响全世界以后，对于应对危机，我们国内的绝大多数专家对此都异口同声，观点基本雷同，而且又莫衷一是，人云亦云，本书的作者以其独特的视觉和丰富的知识，在书中告诉我们这次危机的来龙去脉，我们国家应该以怎样的方式去直面这次危机，如何转危为安，化危为机；作为普通读者也能从书中领会到一些对自己有益的知识。读完这本书以后，你更能体会到一种不一样的思维，非常值得一读。	gold label: 1	score: -67.85609436035156
text: 我从06年开始几乎每月都有一次出差，山西很多酒店都住过了，印象最深的就是三晋国际，其他还有龙城国际，华苑宾馆，黄河京

Using Euclidean distance:

In [11]:
sim_fn = "euc"

res = feature_sim.interpret(test_dataloader, sample_num=sample_num, sim_fn=sim_fn)

print_result(test_data, train_ds, res, data_name='chnsenticorp')

Extracting feature for given dataloader, it will take some time...
test data
text: 没有光驱,重装Windows需要外接光驱,对于电脑新手会很麻烦(没什么人会用Linux吧)	predict label: 0
examples with positive influence
text: Linux系统不太好用,平时习惯用Windows xp 系统,一下子用这个系统感觉很不习惯,建议开发或预装Windows xp系统.	gold label: 0	score: -8.115217208862305
text: 1、机器较沉 2、VISTA用起来不习惯，且占系统较多 3、音频插口、右前侧的二个USB口在用鼠标时感觉手靠得太近了	gold label: 0	score: -8.313434600830078
text: vista系统下也没有无线网卡驱动，用驱动精灵可解决。 机器稍有点重。 散热有待改进。	gold label: 0	score: -8.38914966583252
examples with negative influence
text: “任务型教学”在我国外语教学界备受关注。国家教育部新《英语课程标准》将“倡导‘任务型’的教学途径，培养学生综合语言运用能力”写入教学建议。任务型教学被视为改革我国传统外语教学的良方。本书立足我国外语教学现状，全面分析了“任务型教学”的理论和实践基础、以实例说明“任务型教学”的具体操作步骤。为广大一线英语教师提供了教学和研究参考。	gold label: 1	score: -34.38697814941406
text: 当美国发生次贷危机乃至影响全世界以后，对于应对危机，我们国内的绝大多数专家对此都异口同声，观点基本雷同，而且又莫衷一是，人云亦云，本书的作者以其独特的视觉和丰富的知识，在书中告诉我们这次危机的来龙去脉，我们国家应该以怎样的方式去直面这次危机，如何转危为安，化危为机；作为普通读者也能从书中领会到一些对自己有益的知识。读完这本书以后，你更能体会到一种不一样的思维，非常值得一读。	gold label: 1	score: -34.51872634887695
text: 我从06年开始几乎每月都有一次出差，山西很多酒店都住过了